In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing

In [2]:
data = pd.read_csv("Data//Preprocessed Data//Preprocessed_data.csv")

In [3]:
data.head()

,Unnamed: 0,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,...,RAIN,wd,WSPM,station,PM25_AQI,PM10_AQI,SO2_AQI,NO2_AQI,CO_AQI,O3_AQI
0,0,1,2013,3,1,0,4.0,4.0,1.526718,3.723404,...,0.0,329.976194,4.4,Aotizhongxin,16.666667,3.636364,2.120441,3.447597,2.911208,30.8
1,1,2,2013,3,1,1,8.0,8.0,1.526718,3.723404,...,0.0,350.471596,4.7,Aotizhongxin,33.333333,7.272727,2.120441,3.447597,2.911208,30.8
2,2,3,2013,3,1,2,7.0,7.0,1.908397,5.319149,...,0.0,329.976194,5.6,Aotizhongxin,29.166667,6.363636,2.650551,4.925138,2.911208,29.2
3,3,4,2013,3,1,3,6.0,6.0,4.198473,5.851064,...,0.0,314.006221,3.1,Aotizhongxin,25.000000,5.454545,5.831213,5.417652,2.911208,28.8
4,4,5,2013,3,1,4,3.0,3.0,4.580153,6.382979,...,0.0,350.471596,2.0,Aotizhongxin,12.500000,2.727273,6.361323,5.910165,2.911208,28.8


# Preparing Train and Test Set 

In [4]:
temp_array = data['TEMP'].values
y = temp_array.reshape(-1,1) # Because only one attribute has to be predicted

In [5]:
features = ['PM2.5','PM10','SO2','NO2','O3','CO','PRES','RAIN','WSPM']
X = data.loc[:, features].values

In [6]:
features_aqi = ['PM25_AQI','PM10_AQI','SO2_AQI','NO2_AQI','O3_AQI','CO_AQI','PRES','RAIN','WSPM']

X_aqi = data.loc[:, features_aqi].values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train_aqi, X_test_aqi, y_train_aqi, y_test_aqi = train_test_split(X_aqi, y, test_size = 0.2, random_state = 0)


## DecisionTree Regression

In [8]:
from sklearn.tree import DecisionTreeRegressor

dtree = DecisionTreeRegressor()
dtree_aqi = DecisionTreeRegressor()
dtree.fit(X_train, y_train)
dtree_aqi.fit(X_train_aqi, y_train_aqi)

print(" R^2 Score <-- on test set: {}".format(dtree.score(X_test, y_test)))

print(" R^2 Score with features <-- on test set: {}".format(dtree_aqi.score(X_test_aqi, y_test_aqi)))


 R^2 Score <-- on test set: 0.7250546089852411
 R^2 Score with features <-- on test set: 0.7253724876936607


## Ridge Regression 

In [9]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 1.0, tol = 1, solver = 'svd').fit(X_train, y_train)
ridge_aqi = Ridge(alpha = 1.0, tol = 1, solver = 'svd').fit(X_train_aqi, y_train_aqi)

print(" R^2 Score <-- on test set: {}".format(ridge.score(X_test, y_test)))

print(" R^2 Score with features <-- on test set: {}".format(ridge_aqi.score(X_test_aqi, y_test_aqi)))


 R^2 Score <-- on test set: 0.7502428511220475
 R^2 Score with features <-- on test set: 0.7507785142635952


## Lasso Regression


In [10]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1.0, precompute = False,selection = 'random',tol = 0.000001)
lasso.fit(X_train,y_train)

lasso_aqi = Lasso(alpha=1.0, precompute = False,selection = 'random',tol = 0.000001)
lasso_aqi.fit(X_train_aqi,y_train_aqi)

print(" R^2 Score <-- on test set: {}".format(lasso.score(X_test, y_test)))

print(" R^2 Score with features <-- on test set: {}".format(lasso_aqi.score(X_test_aqi, y_test_aqi)))

 R^2 Score <-- on test set: 0.743782038753257
 R^2 Score with features <-- on test set: 0.7502508584525543


## ElasticNet Regression

In [11]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet().fit(X_train,y_train)
enet_aqi = ElasticNet().fit(X_train_aqi,y_train_aqi)

print(" R^2 Score <-- on test set: {}".format(enet.score(X_test, y_test)))

print(" R^2 Score with features <-- on test set: {}".format(enet_aqi.score(X_test_aqi, y_test_aqi)))

 R^2 Score <-- on test set: 0.7439282443449935
 R^2 Score with features <-- on test set: 0.7504926807317898
